# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

W0809 13:40:11.451142 140657502897984 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/nisreen/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0809 13:40:11.464817 140657502897984 deprecation_wrapper.py:119] From /home/nisreen/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:
hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob")

In [10]:
bob._objects

{}

In [9]:
bob._objects

{}

In [11]:
x = th.tensor([1,2,3,4,5])

In [12]:
x = x.send(bob)

In [13]:
bob._objects

{55145598183: tensor([1, 2, 3, 4, 5])}

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.id_at_location

55145598183

In [16]:
x.id

57839301900

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [19]:
x

(Wrapper)>[PointerTensor | me:57839301900 -> bob:55145598183]

In [19]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [20]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [21]:
alice = sy.VirtualWorker(hook, id="alice")

In [22]:
x = th.tensor([1,2,3,4,5])

In [23]:
x_ptr = x.send(bob, alice)

In [24]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [28]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [29]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [30]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [31]:
x

(Wrapper)>[PointerTensor | me:8684704220 -> bob:16657567066]

In [32]:
y

(Wrapper)>[PointerTensor | me:73257410540 -> bob:54562145831]

In [33]:
z = x + y

In [34]:
z

(Wrapper)>[PointerTensor | me:97524104529 -> bob:77194761341]

In [35]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [36]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:49198139123 -> bob:28383574411]

In [37]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [38]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [39]:
z = (x + y).sum()

In [40]:
z.backward()

(Wrapper)>[PointerTensor | me:83022271641 -> bob:74682013035]

In [41]:
x = x.get()

In [42]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [43]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [44]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [45]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [47]:
pred

(Wrapper)>[PointerTensor | me:56877665583 -> bob:27987177729]

In [49]:
for i in range(10):
    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)


tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [50]:
bob = bob.clear_objects()

In [51]:
bob._objects

{}

In [52]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{75468545691: tensor([1, 2, 3, 4, 5])}

In [54]:
del x

In [55]:
bob._objects

{}

In [56]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [57]:
bob._objects

{57590772950: tensor([1, 2, 3, 4, 5])}

In [58]:
x = "asdf"

In [59]:
bob._objects

{}

In [60]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [61]:
x

(Wrapper)>[PointerTensor | me:46116659357 -> bob:28469374231]

In [62]:
bob._objects

{28469374231: tensor([1, 2, 3, 4, 5])}

In [63]:
x = "asdf"

In [64]:
bob._objects

{28469374231: tensor([1, 2, 3, 4, 5])}

In [65]:
del x

In [66]:
bob._objects

{28469374231: tensor([1, 2, 3, 4, 5])}

In [67]:
bob = bob.clear_objects()
bob._objects

{}

In [68]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [69]:
bob._objects

{69383115049: tensor([1, 2, 3, 4, 5])}

In [70]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [71]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:84130630058 -> bob:96501007099]
Tensor B: tensor([1, 1, 1, 1, 1])

In [72]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [73]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [74]:
from torch import nn, optim

In [75]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [76]:
# A Toy Model
model = nn.Linear(2,1)

In [77]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [78]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.5743)
tensor(0.4514)
tensor(0.2670)
tensor(0.1845)
tensor(0.1308)
tensor(0.0936)
tensor(0.0675)
tensor(0.0490)
tensor(0.0358)
tensor(0.0263)
tensor(0.0194)
tensor(0.0144)
tensor(0.0107)
tensor(0.0080)
tensor(0.0060)
tensor(0.0045)
tensor(0.0034)
tensor(0.0026)
tensor(0.0020)
tensor(0.0015)


In [79]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [80]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [81]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [82]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [83]:
train()

tensor(0.1671, requires_grad=True)
tensor(1.4458, requires_grad=True)
tensor(0.4662, requires_grad=True)
tensor(0.8088, requires_grad=True)
tensor(0.2958, requires_grad=True)
tensor(0.4823, requires_grad=True)
tensor(0.1745, requires_grad=True)
tensor(0.2905, requires_grad=True)
tensor(0.1029, requires_grad=True)
tensor(0.1761, requires_grad=True)
tensor(0.0611, requires_grad=True)
tensor(0.1076, requires_grad=True)
tensor(0.0366, requires_grad=True)
tensor(0.0663, requires_grad=True)
tensor(0.0221, requires_grad=True)
tensor(0.0412, requires_grad=True)
tensor(0.0136, requires_grad=True)
tensor(0.0259, requires_grad=True)
tensor(0.0084, requires_grad=True)
tensor(0.0165, requires_grad=True)
tensor(0.0053, requires_grad=True)
tensor(0.0106, requires_grad=True)
tensor(0.0035, requires_grad=True)
tensor(0.0069, requires_grad=True)
tensor(0.0023, requires_grad=True)
tensor(0.0045, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0010, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [84]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

<VirtualWorker id:alice #tensors:0>

In [85]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [86]:
x = x.send(alice)

In [87]:
bob._objects

{44370016399: tensor([1, 2, 3, 4, 5])}

In [88]:
alice._objects

{98524358853: (Wrapper)>[PointerTensor | alice:98524358853 -> bob:44370016399]}

In [89]:
y = x + x

In [90]:
y

(Wrapper)>[PointerTensor | me:54622141222 -> alice:21824430494]

In [91]:
bob._objects

{44370016399: tensor([1, 2, 3, 4, 5]),
 76768216882: tensor([ 2,  4,  6,  8, 10])}

In [92]:
alice._objects

{98524358853: (Wrapper)>[PointerTensor | alice:98524358853 -> bob:44370016399],
 21824430494: (Wrapper)>[PointerTensor | alice:21824430494 -> bob:76768216882]}

In [93]:
jon = sy.VirtualWorker(hook, id="jon")

In [186]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [187]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [188]:
alice._objects

{40541026396: (Wrapper)>[PointerTensor | alice:40541026396 -> bob:10539507281]}

In [94]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:98524358853 -> bob:44370016399]

In [95]:
bob._objects

{44370016399: tensor([1, 2, 3, 4, 5]),
 76768216882: tensor([ 2,  4,  6,  8, 10])}

In [96]:
alice._objects

{21824430494: (Wrapper)>[PointerTensor | alice:21824430494 -> bob:76768216882]}

In [97]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [98]:
bob._objects

{76768216882: tensor([ 2,  4,  6,  8, 10])}

In [99]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [100]:
bob._objects

{97477923991: tensor([1, 2, 3, 4, 5])}

In [101]:
alice._objects

{16947190458: (Wrapper)>[PointerTensor | alice:16947190458 -> bob:97477923991]}

In [102]:
del x

In [103]:
bob._objects

{}

In [104]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [105]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [106]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [107]:
bob._objects

{81471006749: tensor([1, 2, 3, 4, 5])}

In [108]:
alice._objects

{}

In [109]:
x.move(alice)

(Wrapper)>[PointerTensor | me:83425839913 -> alice:83425839913]

In [110]:
bob._objects

{}

In [111]:
alice._objects

{83425839913: tensor([1, 2, 3, 4, 5])}

In [112]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [113]:
bob._objects

{90689948959: tensor([1, 2, 3, 4, 5])}

In [114]:
alice._objects

{83425839913: tensor([1, 2, 3, 4, 5]),
 55350242576: (Wrapper)>[PointerTensor | alice:55350242576 -> bob:90689948959]}

In [115]:
x.remote_get()

(Wrapper)>[PointerTensor | me:9093407956 -> alice:55350242576]

In [116]:
bob._objects

{}

In [117]:
alice._objects

{83425839913: tensor([1, 2, 3, 4, 5]), 55350242576: tensor([1, 2, 3, 4, 5])}

In [118]:
x.move(bob)

(Wrapper)>[PointerTensor | me:9093407956 -> bob:9093407956]

In [119]:
x

(Wrapper)>[PointerTensor | me:9093407956 -> bob:9093407956]

In [120]:
bob._objects

{9093407956: tensor([1, 2, 3, 4, 5])}

In [121]:
alice._objects

{83425839913: tensor([1, 2, 3, 4, 5])}